In [102]:
# import visualization packages
from polygon import RESTClient
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.express as px

# import pandas and datetime
import pandas_market_calendars as mcal
import pandas as pd
import datetime as dt

# import API_KEY from config
from config import *

In [103]:
client = RESTClient(API_KEY)

In [240]:
ticker = 'AAPL'
earnings_date = '2022-10-27'
expiration_date = '2022-11-04'

In [105]:
# get the date that is 30 days before expiration
start_date = (pd.to_datetime(earnings_date) - dt.timedelta(days=30)).date().strftime('%Y-%m-%d')

In [106]:
# create a dataframe of contracts available as of the date specified in 'contracts_as_of'
contract_names = []
for c in client.list_options_contracts(ticker, limit=1000, as_of=start_date):
    contract_names.append(c)

df = []
for c in contract_names:
    df.append({
        'expiration date': c.expiration_date,
        'type': c.contract_type,
        'strike price': c.strike_price,
        'option ticker': c.ticker
    })
    
df = pd.DataFrame(df)
df['expiration date'] = pd.to_datetime(df['expiration date'])

In [118]:
# get the price of the underlying
underlying = client.get_aggs(ticker=ticker, from_=start_date, to=earnings_date, multiplier=1, timespan='day')
underlying = pd.DataFrame(underlying)
underlying['date'] = pd.to_datetime(underlying['timestamp'] * 1000000)

In [258]:
expiration_mask = df['expiration date'] == expiration_date
call_mask = df['type'] == 'call'
put_mask = df['type'] == 'put'
min_value_mask = round(abs(df[expiration_mask & call_mask]['strike price'] - underlying['close'].iloc[0]).min(), 2)

In [255]:
stock_price = underlying['close'].iloc[0]
df.iloc[abs(df[expiration_mask]['strike price'] - stock_price).sort_values().head(2).index]

,expiration date,type,strike price,option ticker
649,2022-11-04,call,152.5,O:AAPL221104C00152500
702,2022-11-04,put,152.5,O:AAPL221104P00152500


In [271]:
call = df.iloc[abs(df[expiration_mask & call_mask]['strike price'] - stock_price).sort_values().head(1).index]
put = df.iloc[abs(df[expiration_mask & put_mask]['strike price'] - stock_price).sort_values().head(1).index]

In [280]:
def get_net_pnl(call_df, put_df):
    call = client.get_aggs(
        ticker = call_df['option ticker'].iloc[0],
        limit=50000,
        multiplier = 1, 
        timespan = 'day', 
        from_ = start_date, 
        to = earnings_date)
    put = client.get_aggs(
        ticker = put_df['option ticker'].iloc[0],
        limit=50000,
        multiplier = 1, 
        timespan = 'day', 
        from_ = start_date, 
        to = earnings_date)
    call = pd.DataFrame(call)
    call['date'] = pd.to_datetime(call['timestamp'] * 1000000)

    put = pd.DataFrame(put)
    put['date'] = pd.to_datetime(put['timestamp'] * 1000000)
    
    net_df = pd.DataFrame()
    net_df['close'] = call['close'] + put['close']
    net_df['date'] = call['date']
    return net_df

In [281]:
net_df = get_net_pnl(call, put)

In [282]:
net_df

,close,date
0,15.97,2022-09-27 04:00:00
1,15.33,2022-09-28 04:00:00
2,17.43,2022-09-29 04:00:00
3,17.09,2022-09-30 04:00:00
4,14.90,2022-10-03 04:00:00
5,14.05,2022-10-04 04:00:00
6,15.40,2022-10-05 04:00:00
7,13.82,2022-10-06 04:00:00
8,15.97,2022-10-07 04:00:00
9,15.62,2022-10-10 04:00:00


# Find valid trading dates

In [77]:
trading_days = []
pd.date_range(start_date, earnings_date)

DatetimeIndex(['2022-09-27', '2022-09-28', '2022-09-29', '2022-09-30',
               '2022-10-01', '2022-10-02', '2022-10-03', '2022-10-04',
               '2022-10-05', '2022-10-06', '2022-10-07', '2022-10-08',
               '2022-10-09', '2022-10-10', '2022-10-11', '2022-10-12',
               '2022-10-13', '2022-10-14', '2022-10-15', '2022-10-16',
               '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20',
               '2022-10-21', '2022-10-22', '2022-10-23', '2022-10-24',
               '2022-10-25', '2022-10-26', '2022-10-27'],
              dtype='datetime64[ns]', freq='D')

In [85]:
for date in pd.date_range(start_date, earnings_date):
    if date.dayofweek != 5 and date.dayofweek != 6:
        print(date)

2022-09-27 00:00:00
2022-09-28 00:00:00
2022-09-29 00:00:00
2022-09-30 00:00:00
2022-10-03 00:00:00
2022-10-04 00:00:00
2022-10-05 00:00:00
2022-10-06 00:00:00
2022-10-07 00:00:00
2022-10-10 00:00:00
2022-10-11 00:00:00
2022-10-12 00:00:00
2022-10-13 00:00:00
2022-10-14 00:00:00
2022-10-17 00:00:00
2022-10-18 00:00:00
2022-10-19 00:00:00
2022-10-20 00:00:00
2022-10-21 00:00:00
2022-10-24 00:00:00
2022-10-25 00:00:00
2022-10-26 00:00:00
2022-10-27 00:00:00


In [86]:
nyse = mcal.get_calendar('NYSE')

In [99]:
valid_dates = nyse.valid_days(start_date=start_date, end_date=earnings_date)
valid_dates = [date.date().strftime('%Y-%m-%d') for date in valid_dates]